In [305]:
import numpy as np
import graph
from collections import defaultdict

## Preparation Functions/Classes

In [325]:
def read_instance_file(filename):
    '''
    ::param: filename: string, filename that indicates the location of instance data file
    ::return value: (specification, data)
    :: specification: dict, specification of the instance
    :: data: the numpy array with a list of edges and their cost, demand
    :: data: [vertex1 vertex2 cost demand]
    '''
    fd = open(inputfile)
    content = fd.readlines()
    content = [x.strip() for x in content] 
    specification = dict()
    for i in range(8):
        line = content[i].split(':')
        specification[line[0].strip()] = line[1].strip()
    # print(specification)
    data = list()
    for line in content[9:-1]:
        tmp = line.split()
        data.append([int(x.strip()) for x in tmp])
    data = np.array(data)
    fd.close()
    return specification, data

In [411]:
'''
filelist CARP_samples
egl-e1-A.dat  gdb10.dat  val1A.dat  val7A.dat
egl-s1-A.dat  gdb1.dat   val4A.dat
'''
inputfile = 'CARP_samples/egl-s1-A.dat'

In [412]:
# reload graph.py
reload(graph)

<module 'graph' from 'graph.pyc'>

## Read the graph into Data Structure

In [413]:
spec, data = read_instance_file(inputfile)
gf = graph.Graph()
gf.load_from_data(data.tolist())

In [414]:
print(gf)
print(spec)
print(gf.get_tasks())

vertices: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 
edges: set([1, 116]) set([2, 117]) set([128, 3]) set([4, 124]) set([5, 6]) set([8, 6]) set([6, 7]) set([8, 9]) set([8, 11]) set([9, 10]) set([10, 19]) set([10, 15]) set([33, 11]) set([11, 27]) set([11, 12]) set([12, 20]) set([12, 13]) set([16, 13]) set([17, 13]) set([13, 14]) set([16, 15]) set([17, 15]) set([17, 18]) set([18, 19]) set([24, 20]) set([20, 21]) set([20, 22]) set([21, 23]) set([24, 23]) set([26, 23]) set([24, 25]) set([25, 26]) set([25, 27]) set([26, 31]) set([27, 28]) set([28, 29]) set([28, 30]) set([33, 29]) set([32,

In [415]:
capcacity = int(spec['CAPACITY'])
depot = int(spec['DEPOT'])

In [416]:
# test
gf.get_shortest_path(1,1)

([], 0)

In [417]:
gf[2]

{117: {'cost': 16, 'demand': 16}}

## Initialization
### Path-Scanning Simple

In [418]:
def which_better(u1, u2, graph, load, capacity, depot):
    r_cq1 = graph[u1[0]][u1[1]]['cost']/graph[u1[0]][u1[1]]['demand']
    r_cq2 = graph[u2[0]][u2[1]]['cost']/graph[u2[0]][u2[1]]['demand']
    return_cost1 = graph.get_shortest_path(u1[1], depot)[1]
    return_cost2 = graph.get_shortest_path(u2[1], depot)[1]
    if load < capacity/2:
        if r_cq1 > r_cq2:
            return u1
        elif r_cq1 < r_cq2:
            return u2
        elif return_cost1 > return_cost2:
            return u1
        elif return_cost1 < return_cost2:
            return u2
        else:
            import random
            return random.choice([u1, u2])
    else:
        if r_cq1 < r_cq2:
            return u1
        elif r_cq1 > r_cq2:
            return u2
        elif return_cost1 < return_cost2:
            return u1
        elif return_cost1 > return_cost2:
            return u2
        else:
            import random
            return random.choice([u1, u2])

def path_scanning(graph, depot):
    k = 0
    R = defaultdict(dict)
    load = defaultdict(dict)
    cost = defaultdict(dict)
    free_task = set(graph.get_tasks())
    while len(free_task) > 0:
        k += 1
        R[k] = list()
        load[k], cost[k] = 0, 0
        end = depot
        u = None
        #print("############ beign #############")
        while True:
            if len(free_task) == 0:
                break
            d_min = float('inf')
            for f_task in free_task:
                if graph[f_task[0]][f_task[1]]['demand'] + load[k] > capcacity:
                    continue
                if u == None:
                    u = f_task
                d_tmp = graph.get_shortest_path(end, f_task[0])[1]
                # print(d_tmp,d_min, end,f_task, u)
                if d_tmp < d_min:
                    d_min = d_tmp
                    u = f_task
                elif d_tmp == d_min:
                    d_min = d_tmp
                    #print(u)
                    u = which_better(u, f_task, gf, load[k], capcacity, depot)
                    #print(u)
            if d_min == float('inf'):
                break
            #if u != None:
            R[k].append(u)
            free_task.remove(u)
            free_task.remove((u[1],u[0]))
            cost[k] += graph[u[0]][u[1]]['cost'] + d_min
            load[k] += graph[u[0]][u[1]]['demand']
            end = u[1]
        cost[k] += graph.get_shortest_path(u[1], depot)[1]
        #print("############ end #############")
    return R, load, cost

In [419]:
path_scanning(gf, 1)

(defaultdict(dict,
             {1: [(1, 116),
               (116, 117),
               (117, 119),
               (117, 2),
               (118, 114),
               (114, 113),
               (113, 112),
               (112, 107),
               (107, 110),
               (110, 112),
               (110, 111),
               (106, 105),
               (97, 98)],
              2: [(107, 106),
               (105, 104),
               (104, 102),
               (66, 62),
               (62, 63),
               (63, 64),
               (64, 65),
               (56, 55),
               (55, 54),
               (55, 140),
               (140, 49),
               (49, 48),
               (139, 33),
               (12, 13)],
              3: [(107, 108),
               (108, 109),
               (66, 67),
               (67, 68),
               (67, 69),
               (69, 71),
               (71, 72),
               (72, 73)],
              4: [(87, 86),
               (86, 85),
        

## Metaheuristics

## Solution-related

### output example

```
s 0,(1,2),(2,4),(4,1),0,0,(4,3),(3,1),0
q 15
```

In [250]:
float('inf')<1

False

In [274]:
x = [1,2]

In [289]:
x[0:2:1]

[1, 2]